In [1]:

import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


2023-07-24 17:13:22.958157: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 17:13:23.703883: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:


inputs=tf.keras.layers.Input(shape=(28, 28), name='input')
#lstm= tf.keras.layers.LSTM(20, return_sequences=True)
lstmcell=tf.keras.layers.LSTMCell(20)
lstm = tf.keras.layers.RNN(lstmcell,return_sequences=True)

x=tf.keras.layers.Bidirectional(lstm)(inputs)

x=tf.keras.layers.Flatten()(x)
outputs=tf.keras.layers.Dense(10, activation=tf.nn.softmax, name='output')(x)
model=tf.keras.Model(inputs=inputs,outputs=outputs)


model.summary()





Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 28, 28)]          0         
                                                                 
 bidirectional (Bidirection  (None, 28, 40)            7840      
 al)                                                             
                                                                 
 flatten (Flatten)           (None, 1120)              0         
                                                                 
 output (Dense)              (None, 10)                11210     
                                                                 
Total params: 19050 (74.41 KB)
Trainable params: 19050 (74.41 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-07-24 17:13:24.556089: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-24 17:13:24.556117: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: r2d2
2023-07-24 17:13:24.556122: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: r2d2
2023-07-24 17:13:24.556170: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.54.3
2023-07-24 17:13:24.556185: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.54.3
2023-07-24 17:13:24.556190: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.54.3


In [3]:
class InferModel(tf.Module):
    def __init__(self, model):
        super().__init__()

        self.model = model

    @tf.function(
        input_signature=[tf.TensorSpec(shape=(1,28,28), dtype=tf.float32, name="inputs")]
    )
    def __call__(self, inputs):
        return self.model(inputs)
        #return {"outputs": outputs}

MODEL_DIR = "keras_lstm"
infer=InferModel(model)
tf.saved_model.save(infer,MODEL_DIR)


INFO:tensorflow:Assets written to: keras_lstm/assets


INFO:tensorflow:Assets written to: keras_lstm/assets


In [4]:

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


2023-07-24 17:13:27.065286: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-07-24 17:13:27.065310: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-07-24 17:13:27.065844: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: keras_lstm
2023-07-24 17:13:27.069876: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-07-24 17:13:27.069888: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: keras_lstm
2023-07-24 17:13:27.077494: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-07-24 17:13:27.079720: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-07-24 17:13:27.108084: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: keras_lstm
2023-07-24 17:13:27.12684